In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import torch
from PIL import Image
from torchvision import transforms
import imgproc
from torchvision.transforms.functional import InterpolationMode as IMode
from Model.aetad import AETAD_4Direct
import torch.nn.functional as F

In [2]:
model = AETAD_4Direct()
checkpoint = torch.load('./jupyter_pth/aeted/g-best.pth', map_location='cuda:0')
checkpoint = {k.replace('module.',''):v for k,v in checkpoint.items()}
# model = torch.nn.DataParallel(model)
model.load_state_dict(checkpoint)
model.eval()

AETAD_4Direct(
  (_downscaling): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): _ReversePixelShuffle_(downscale_factor=4)
  )
  (_res_en1): _Resblock_(
    channels=64
    (filter_block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (_res_en2): _Resblock_(
    channels=64
    (filter_block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (_res_en3): _Resblock_(
    channels=64
    (filter_block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size

In [3]:
image_dir = './jupyter_src/pic/0822.png'
image = Image.open(image_dir)
# hr_transforms = transforms.CenterCrop(96)
hr_image = image
lr_transforms = transforms.Resize([hr_image.size[1]//4, hr_image.size[0]//4] , interpolation=IMode.BICUBIC)

print(hr_image.size)
hr_image.save('./jupyter_res/pic/origin.png')
hr_image.show('Original')
lr_image = lr_transforms(hr_image)
lr_image.show("BICUBIC")
lr_image.save('./jupyter_res/pic/bicubic.png')

(2040, 1596)


In [5]:
hr_tensor = imgproc.image2tensor(hr_image, range_norm=False, half=False)
lr_tensor = imgproc.image2tensor(lr_image, range_norm=False, half=False)
print(hr_tensor.shape)
with torch.no_grad():
    hr_tensor = hr_tensor.reshape(1,3,1596,2040)
    lr_tensor = lr_tensor.reshape(1,lr_tensor.shape[0], lr_tensor.shape[1], lr_tensor.shape[2])
    down = model.encode(hr_tensor)
    reveal = model.forward(hr_tensor)
    print(down)
    down_img = Image.fromarray(imgproc.tensor2image(F.normalize(-down, p=2, dim=1) , range_norm=False, half=False))
    reveal_img = Image.fromarray(imgproc.tensor2image(reveal, range_norm=False, half=False))
    down_img.show('down_result')
    reveal_img.show('reveal_result')
    down_img.save('./jupyter_res/pic/feature.png')
    reveal_img.save('./jupyter_res/pic/reveal.png')

torch.Size([3, 1596, 2040])
tensor([[[[-0.6297, -0.4376, -0.4379,  ..., -0.0768, -0.0608, -0.0952],
          [-0.2343, -0.1259, -0.1129,  ...,  0.0040, -0.0063, -0.0272],
          [-0.2785, -0.1826, -0.1844,  ...,  0.0047, -0.0054, -0.0165],
          ...,
          [-0.1604, -0.2137, -0.2321,  ..., -0.2571, -0.3173, -0.2363],
          [-0.2705, -0.2138, -0.2395,  ...,  0.2368, -0.1646, -0.4273],
          [-0.0780, -0.2352, -0.2815,  ...,  0.0117, -0.2528, -0.2609]],

         [[ 0.0992, -0.0558, -0.1085,  ..., -0.0316, -0.0403, -0.0079],
          [ 0.0280, -0.1632, -0.1688,  ..., -0.0486, -0.0432, -0.0202],
          [ 0.0233, -0.1378, -0.1503,  ..., -0.0590, -0.0497, -0.0299],
          ...,
          [-0.4534,  0.0018, -0.0100,  ...,  0.1248,  0.3744, -0.0443],
          [-0.1614,  0.0139, -0.0783,  ..., -0.0947, -0.1627,  0.0208],
          [-0.0245, -0.0442,  0.0838,  ..., -0.1062, -0.1327,  0.1078]],

         [[-0.4043, -0.3809, -0.3187,  ..., -0.0192, -0.0268, -0.0303],
  